<a href="https://colab.research.google.com/github/Team-Takeoff/Capstone/blob/Dave/data_acquisition/FAA_Aircraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FAA Aircraft Code

This pulls the tail numbers from our dataset and attempts to get information from the faa registry site. Using selenium, only so many tail numbers could be passed at a time.

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
import json

In [ ]:
path_to_files = ''
files = json.load(path_to_files,'r')
df = pd.read_csv(files['Aircraft']['All_Tails'])
df.columns = ['TailNo']
tail_numbers = list(df.TailNo.head())


In [ ]:
def get_aircraft_data(n_number):
    # Configure Selenium WebDriver (assuming Chrome driver is installed)
    driver = webdriver.Safari()
    driver.delete_all_cookies()


    # Construct the URL for the Aircraft Inquiry - N-Number Search page
    url = f"https://registry.faa.gov/aircraftinquiry/Search/NNumberInquiry"

    # Navigate to the URL
    driver.get(url)
    wait = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.NAME, 'NNumbertxt')))

    # Find the table row containing the manufacturing year
    field = driver.find_element(By.NAME, 'NNumbertxt')
    field.send_keys(n_number)

    form = driver.find_element(By.TAG_NAME, 'form')
    form.submit()

    wait = WebDriverWait(driver, 19)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'tbody')))
    page_content = driver.page_source

    soup = BeautifulSoup(page_content, 'html.parser')
    driver.quit()
    return soup


aircraft_file = files['Aircraft']['FAA_Tails']
with open(aircraft_file, 'a', buffering=-1) as file:
    for n_number in tail_numbers:
        # the web page validation checks: starting with N and 4th character is numeric
        if n_number[0] == 'N' and n_number[3].isdigit() == True and n_number[1] != '0':
            soup = get_aircraft_data(n_number)
            yr = soup.find_all(attrs={'data-label': 'Mfr Year' })
            if yr:
                mfr_yr = yr[0].get_text().strip()
            else:
                yr = soup.find_all(attrs={'data-label' : 'Year Manufacturer'})
                if yr:
                    mfr_yr = yr[0].get_text().strip()
                else:
                    mfr_yr = ''
            mfr = soup.find_all(attrs={'data-label': 'Manufacturer Name' })
            if mfr:
                mfr_text = mfr[0].get_text().strip()
            else:
                mfr_text = ""
            model = soup.find_all(attrs={'data-label': 'Model' })
            if model:
                model_txt = model[0].get_text().strip()
            else:
                model_txt = ""

            file.write(n_number + ',' + mfr_yr + ',' + mfr_text + ',' + model_txt + '\n')
            time.sleep(np.random.randint(12,30))
        else:
            file.write(n_number + ',,,\n')


In [ ]:
rd = pd.read_csv(files['Aircraft']['FAA_Tails'])

In [ ]:
rd.head()

,Tail_NO,Year_Built,Manufacturer,Model
0,N301DQ,None,BOEING,737-732
1,N554NW,2001,BOEING,757-251
2,N760NC,1976,MCDONNELL DOUGLAS,DC-9-51
3,N956DL,1990,MCDONNELL DOUGLAS AIRCRAFT CO,MD-88
4,N394DA,2000,BOEING,737-832
